# Total Portfolio Risk Analysis

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import math 

pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

### Portfolio Builder

In [ ]:
portfolio = pd.DataFrame(columns=["ticker","style","type","strike","maturity","quantity","spot","sigma","q_div"])

def add_instrument(ticker, style, opt_type, strike, maturity, quantity):
    spot, q = fetch_spot_and_dividend_yield(ticker)
    rets = fetch_returns([ticker])
    sigma = float(rets[ticker].std() * math.sqrt(252))
    new_row = dict(ticker=ticker, style=style, type=opt_type,
                   strike=strike, maturity=maturity, quantity=quantity,
                   spot=spot, sigma=sigma, q_div=q)
    global portfolio
    portfolio = pd.concat([portfolio, pd.DataFrame([new_row])], ignore_index=True)
    print(f"Added {ticker} | spot={spot:.2f} | sigma={sigma:.2f} | q={q:.3f}")

### Helper functions

#### European options

In [ ]:
def bs_d1_d2(S, K, r, q, sigma, T): ### BS Model pricing
    if S <= 0 or K <= 0 or sigma <= 0 or T <= 0:
        return np.nan, np.nan
    d1 = (math.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return d1, d2

def bs_price_greeks(S, K, r, q, sigma, T, opt_type="Call"): ### Greeks for European
    opt_type = opt_type.lower()
    d1, d2 = bs_d1_d2(S, K, r, q, sigma, T)
    if np.isnan(d1):
        return {k: np.nan for k in ["price","delta","gamma","vega","theta"]}

    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    nd1 = norm.pdf(d1)

    if opt_type == "call":
        price = S*math.exp(-q*T)*Nd1 - K*math.exp(-r*T)*Nd2
        delta = math.exp(-q*T)*Nd1
        theta = (-S*nd1*sigma*math.exp(-q*T)/(2*math.sqrt(T))) - r*K*math.exp(-r*T)*Nd2 + q*S*math.exp(-q*T)*Nd1
    else:
        price = K*math.exp(-r*T)*norm.cdf(-d2) - S*math.exp(-q*T)*norm.cdf(-d1)
        delta = math.exp(-q*T)*(Nd1 - 1)
        theta = (-S*nd1*sigma*math.exp(-q*T)/(2*math.sqrt(T))) + r*K*math.exp(-r*T)*norm.cdf(-d2) - q*S*math.exp(-q*T)*norm.cdf(-d1)

    gamma = math.exp(-q*T)*nd1/(S*sigma*math.sqrt(T))
    vega = S*math.exp(-q*T)*nd1*math.sqrt(T)/100
    return dict(price=price, delta=delta, gamma=gamma, vega=vega, theta=theta/365)


#### American Options

In [ ]:
def crr_binomial_american(S, K, r, q, sigma, T, steps=200, opt_type="Call"): ### Binomial Tree Pricing
    dt = T / steps
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u
    a = math.exp((r - q) * dt)
    p = (a - d) / (u - d)
    p = min(max(p, 0), 1)

    prices = np.array([S * (u ** j) * (d ** (steps - j)) for j in range(steps + 1)])
    values = np.maximum(prices - K, 0) if opt_type.lower() == "call" else np.maximum(K - prices, 0)
    disc = math.exp(-r * dt)

    for i in range(steps - 1, -1, -1):
        prices = prices[:-1] * u
        values = disc * (p * values[1:] + (1 - p) * values[:-1])
        if opt_type.lower() == "call":
            values = np.maximum(values, prices - K)
        else:
            values = np.maximum(values, K - prices)
    return values[0]

def american_price_greeks_fd(S, K, r, q, sigma, T, steps=200, opt_type="Call", bump=0.01): ### Finite Difference Greeks
    base = crr_binomial_american(S, K, r, q, sigma, T, steps, opt_type)
    dS = S * bump
    up = crr_binomial_american(S + dS, K, r, q, sigma, T, steps, opt_type)
    down = crr_binomial_american(S - dS, K, r, q, sigma, T, steps, opt_type)
    delta = (up - down) / (2 * dS)
    gamma = (up - 2 * base + down) / (dS ** 2)

    dsig = sigma * bump
    upv = crr_binomial_american(S, K, r, q, sigma + dsig, T, steps, opt_type)
    vega = (upv - base) / (dsig * 100)

    dT = min(1/365, T/50)
    theta = (crr_binomial_american(S, K, r, q, sigma, T - dT, steps, opt_type) - base) / dT / 365
    return dict(price=base, delta=delta, gamma=gamma, vega=vega, theta=theta)


#### Market Data via yfinance

In [ ]:
def fetch_spot_and_dividend_yield(ticker):
    tk = yf.Ticker(ticker)
    hist = tk.history(period="3y")
    spot = float(hist["Close"].iloc[-1])
    divs = tk.dividends
    if divs is None or len(divs) == 0:
        q = 0
    else:
        q = float(divs[-4:].sum() / spot) 
    return spot, q

def fetch_returns(tickers):
    data = yf.download(tickers, period="3y")["Close"]
    rets = np.log(data/data.shift(1)).dropna()
    return rets

#### Risk Computation and VaR

In [ ]:
def instrument_metrics(row, rf=0.04):
    S,K,T,q,sigma,qty=row["spot"],row["strike"],row["maturity"],row["q_div"],row["sigma"],row["quantity"]
    if row["style"].lower()=="european":
        g=bs_price_greeks(S,K,rf,q,sigma,T,row["type"])
    else:
        g=american_price_greeks_fd(S,K,rf,q,sigma,T,opt_type=row["type"])
    return {k:v*qty for k,v in g.items()}

def compute_portfolio_table():
    res=[]
    for _,r in portfolio.iterrows():
        m=instrument_metrics(r)
        res.append(dict(ticker=r["ticker"],type=r["type"],style=r["style"],strike=r["strike"],mat=r["maturity"],**m))
    df=pd.DataFrame(res)
    total=df[["price","delta","gamma","vega","theta"]].sum()
    df.loc["TOTAL"]=["","","","",0,total["price"],total["delta"],total["gamma"],total["vega"],total["theta"]]
    return df

def historical_var(conf=0.95):
    tickers = list(portfolio["ticker"].unique())
    rets = fetch_returns(tickers)
    pnl=[]
    for date,row in rets.iterrows():
        day_pnl=0
        for _,p in portfolio.iterrows():
            delta=instrument_metrics(p)["delta"]
            S=p["spot"]; r=row[p["ticker"]]
            day_pnl+=delta*S*r
        pnl.append(day_pnl)
    pnl=pd.Series(pnl)
    var=np.quantile(-pnl,conf)
    return var,pnl


#### Run analysis

In [ ]:
def run_risk_analysis():
    table=compute_portfolio_table()
    display(table)
    var,pnl=historical_var()
    print(f"Portfolio PV: ${table.loc['TOTAL','price']:.2f}")
    print(f"Delta: {table.loc['TOTAL','delta']:.2f}, Gamma: {table.loc['TOTAL','gamma']:.4f}, Vega: {table.loc['TOTAL','vega']:.2f}, Theta: {table.loc['TOTAL','theta']:.2f}")
    print(f"95% 1-Day VaR: ${var:.2f}")
    pnl.hist(bins=40); plt.title("Portfolio Daily P&L"); plt.show()


### Run analysis

In [ ]:
### add_instrument(ticker, style, opt_type, strike, maturity, quantity)
# Ticker: AAPL, MSFT, TSLA, SPY
# Style: European or American 
# opt_type: Call or Put 
# strike: strike price 
# maturity: in years 
# quantity: Positive for long and Negative for short 

portfolio = pd.DataFrame(columns=portfolio.columns)  # reset
add_instrument("AAPL","European","Call",100,1.0,100)
add_instrument("MSFT","American","Put",300,3,-50)
add_instrument("TSLA","American","Put",300,0.5,-50)
add_instrument("MSFT","European","Put",300,2,100)
add_instrument("SPY","European","Put",200,5,-250)


run_risk_analysis()